In [1]:
pip install -q scikit-learn imbalanced-learn matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)
from sklearn.manifold import TSNE
from sklearn.svm import SVC


In [3]:
# Update this to your own Google Drive path
base_dir = "E:\AMAR\git\EEG_DATAREAD\edf_analyser\preprocessed_data_1005_multilabels"
features_dir = os.path.join(base_dir, "features")
labels_dir = os.path.join(base_dir, "labels")
save_plot_dir = os.path.join(base_dir, "c_plots")
os.makedirs(save_plot_dir, exist_ok=True)

global_class_map = {
    'rest': 0,
    'real_left_fist': 1,
    'real_right_fist': 2,
    'imagined_left_fist': 3,
    'imagined_right_fist': 4,
    'real_both_fists': 5,
    'real_both_feet': 6,
    'imagined_both_fists': 7,
    'imagined_both_feet': 8
}

def get_class_ids_from_run(run_number):
    if run_number in [3, 7, 11]:
        return {'rest': 'rest', 'real_left_fist': 'real_left_fist', 'real_right_fist': 'real_right_fist'}
    elif run_number in [4, 8, 12]:
        return {'rest': 'rest', 'imagined_left_fist': 'imagined_left_fist', 'imagined_right_fist': 'imagined_right_fist'}
    elif run_number in [5, 9, 13]:
        return {'rest': 'rest', 'real_both_fists': 'real_both_fists', 'real_both_feet': 'real_both_feet'}
    elif run_number in [6, 10, 14]:
        return {'rest': 'rest', 'imagined_both_fists': 'imagined_both_fists', 'imagined_both_feet': 'imagined_both_feet'}
    else:
        return None


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14008\2076455317.py:2: SyntaxWarning: invalid escape sequence '\A'
  base_dir = "E:\AMAR\git\EEG_DATAREAD\edf_analyser\preprocessed_data_1005_multilabels"


In [4]:
def evaluate_model(X, y, task_name, save_dir=None, prefix=None, subject_id=None, run_num=None, results_list=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

    clf = SVC(kernel='rbf', C=1, gamma='scale')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='binary')
    rec = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    print(f"Task: {task_name} | Acc: {acc:.2f} | Prec: {prec:.2f} | Rec: {rec:.2f} | F1: {f1:.2f}")

    if results_list is not None:
        results_list.append({
            "Subject": subject_id,
            "Run": run_num,
            "Task": task_name,
            "Accuracy": round(acc, 4),
            "Precision": round(prec, 4),
            "Recall": round(rec, 4),
            "F1": round(f1, 4)
        })

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix: {task_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    if save_dir and prefix:
        cm_path = os.path.join(save_dir, f"{prefix}_confusion_matrix.png")
        plt.savefig(cm_path)
    plt.close()

    # t-SNE
    if len(X) >= 10:
        try:
            tsne = TSNE(n_components=2, perplexity=min(30, len(X) // 3), init='random', random_state=42)
            X_embedded = tsne.fit_transform(X)
            plt.figure(figsize=(5, 4))
            sns.scatterplot(x=X_embedded[:, 0], y=X_embedded[:, 1], hue=y, palette='Set1')
            plt.title(f"t-SNE: {task_name}")
            if save_dir and prefix:
                tsne_path = os.path.join(save_dir, f"{prefix}_tsne.png")
                plt.savefig(tsne_path)
            plt.close()
        except Exception as e:
            print(f"[{task_name}] t-SNE failed: {e}")


In [5]:
run_results = []

task_definitions = [
    ("Real Left vs Real Right", ['real_left_fist', 'real_right_fist']),
    ("Imagined Left vs Real Left", ['imagined_left_fist', 'real_left_fist']),
    ("Real Right vs Imagined Right", ['real_right_fist', 'imagined_right_fist']),
    ("Imagined Left vs Imagined Right", ['imagined_left_fist', 'imagined_right_fist'])
]

for fname in sorted(os.listdir(features_dir)):
    if not fname.endswith('_features.npy'):
        continue

    base_name = fname.replace('_features.npy', '')
    subj_run_match = re.match(r"(S\d{3})R(\d{2})", base_name)
    if not subj_run_match:
        continue

    subj_id, run_str = subj_run_match.groups()
    run_num = int(run_str)
    class_map = get_class_ids_from_run(run_num)
    if class_map is None:
        continue

    feat_path = os.path.join(features_dir, f"{base_name}_features.npy")
    label_path = os.path.join(labels_dir, f"{base_name}_labels.npy")
    if not os.path.exists(label_path):
        continue

    X = np.load(feat_path)
    y = np.load(label_path)

    for task_name, class_keys in task_definitions:
        if all(k in class_map for k in class_keys):
            ids = [global_class_map[k] for k in class_keys]
            task_mask = np.isin(y, ids)
            X_task = X[task_mask]
            y_task = y[task_mask]
            if len(np.unique(y_task)) != 2:
                continue
            y_task_bin = (y_task == ids[1]).astype(int)
            evaluate_model(
                X_task, y_task_bin,
                f"{subj_id} Run {run_num}: {task_name}",
                save_dir=save_plot_dir,
                prefix=f"{subj_id}_R{run_num}_{task_name.replace(' ', '_')}",
                subject_id=subj_id,
                run_num=run_num,
                results_list=run_results
            )


Task: S001 Run 3: Real Left vs Real Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S001 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S001 Run 7: Real Left vs Real Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S001 Run 8: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S001 Run 11: Real Left vs Real Right | Acc: 0.80 | Prec: 1.00 | Rec: 0.67 | F1: 0.80


e:\e\exe\MNE-Python\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Task: S001 Run 12: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S002 Run 3: Real Left vs Real Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S002 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.67 | F1: 0.57
Task: S002 Run 7: Real Left vs Real Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.33 | F1: 0.40
Task: S002 Run 8: Imagined Left vs Imagined Right | Acc: 0.80 | Prec: 1.00 | Rec: 0.50 | F1: 0.67
Task: S002 Run 11: Real Left vs Real Right | Acc: 0.40 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S002 Run 12: Imagined Left vs Imagined Right | Acc: 0.80 | Prec: 1.00 | Rec: 0.50 | F1: 0.67


e:\e\exe\MNE-Python\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Task: S003 Run 3: Real Left vs Real Right | Acc: 0.40 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S003 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S003 Run 7: Real Left vs Real Right | Acc: 0.80 | Prec: 0.67 | Rec: 1.00 | F1: 0.80
Task: S003 Run 8: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.67 | F1: 0.57
Task: S003 Run 11: Real Left vs Real Right | Acc: 1.00 | Prec: 1.00 | Rec: 1.00 | F1: 1.00
Task: S003 Run 12: Imagined Left vs Imagined Right | Acc: 0.60 | Prec: 0.50 | Rec: 1.00 | F1: 0.67
Task: S004 Run 3: Real Left vs Real Right | Acc: 0.20 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S004 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S004 Run 7: Real Left vs Real Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.33 | F1: 0.40
Task: S004 Run 8: Imagined Left vs Imagined Right | Acc: 0.20 | Prec: 0.33 | Rec: 0.33 | F1: 0.33


...

Task: S100 Run 12: Imagined Left vs Imagined Right | Acc: 0.50 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S101 Run 3: Real Left vs Real Right | Acc: 0.60 | Prec: 0.50 | Rec: 1.00 | F1: 0.67
Task: S101 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.40 | Rec: 1.00 | F1: 0.57
Task: S101 Run 7: Real Left vs Real Right | Acc: 0.60 | Prec: 0.67 | Rec: 0.67 | F1: 0.67
Task: S101 Run 8: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.67 | F1: 0.57
Task: S101 Run 11: Real Left vs Real Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S101 Run 12: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.67 | F1: 0.57
Task: S102 Run 3: Real Left vs Real Right | Acc: 0.40 | Prec: 0.40 | Rec: 1.00 | F1: 0.57
Task: S102 Run 4: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.33 | Rec: 0.50 | F1: 0.40
Task: S102 Run 7: Real Left vs Real Right | Acc: 0.60 | Prec: 0.50 | Rec: 1.00 | F1: 0.67
Task: S102 Run 8: Imagined Left vs Imagined Right | Acc: 

e:\e\exe\MNE-Python\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Task: S103 Run 3: Real Left vs Real Right | Acc: 0.60 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S103 Run 4: Imagined Left vs Imagined Right | Acc: 0.80 | Prec: 0.75 | Rec: 1.00 | F1: 0.86
Task: S103 Run 7: Real Left vs Real Right | Acc: 0.60 | Prec: 0.50 | Rec: 0.50 | F1: 0.50
Task: S103 Run 8: Imagined Left vs Imagined Right | Acc: 0.80 | Prec: 1.00 | Rec: 0.50 | F1: 0.67
Task: S103 Run 11: Real Left vs Real Right | Acc: 0.60 | Prec: 1.00 | Rec: 0.33 | F1: 0.50
Task: S103 Run 12: Imagined Left vs Imagined Right | Acc: 0.40 | Prec: 0.50 | Rec: 0.67 | F1: 0.57
Task: S104 Run 3: Real Left vs Real Right | Acc: 0.20 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S104 Run 4: Imagined Left vs Imagined Right | Acc: 0.20 | Prec: 0.33 | Rec: 0.33 | F1: 0.33
Task: S104 Run 7: Real Left vs Real Right | Acc: 0.80 | Prec: 0.67 | Rec: 1.00 | F1: 0.80
Task: S104 Run 8: Imagined Left vs Imagined Right | Acc: 0.25 | Prec: 0.00 | Rec: 0.00 | F1: 0.00
Task: S104 Run 11: Real Left vs Real Right | Acc: 0.20 | P

e:\e\exe\MNE-Python\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Task: S109 Run 12: Imagined Left vs Imagined Right | Acc: 0.60 | Prec: 0.00 | Rec: 0.00 | F1: 0.00


In [6]:
subject_results = []
subject_files = {}

for fname in sorted(os.listdir(features_dir)):
    if fname.endswith('_features.npy'):
        subj_run_match = re.match(r"(S\d{3})R(\d{2})", fname)
        if subj_run_match:
            subj_id = subj_run_match.group(1)
            subject_files.setdefault(subj_id, []).append(fname.replace('_features.npy', ''))

for subj_id, base_names in subject_files.items():
    all_X, all_y = [], []
    for base_name in base_names:
        feat_path = os.path.join(features_dir, f"{base_name}_features.npy")
        label_path = os.path.join(labels_dir, f"{base_name}_labels.npy")
        if os.path.exists(label_path):
            all_X.append(np.load(feat_path))
            all_y.append(np.load(label_path))
    if not all_X:
        continue

    X_all = np.vstack(all_X)
    y_all = np.concatenate(all_y)

    # Task 5: Real vs Imagined Fists
    fist_ids = [1, 2, 3, 4]
    real_ids = [1, 2]
    imag_ids = [3, 4]
    mask_fist = np.isin(y_all, fist_ids)
    y_realimag = np.where(np.isin(y_all[mask_fist], real_ids), 0, 1)
    evaluate_model(
        X_all[mask_fist], y_realimag,
        f"{subj_id}: Real vs Imagined Fists",
        save_dir=save_plot_dir,
        prefix=f"{subj_id}_Real_vs_Imagined_Fists",
        subject_id=subj_id,
        run_num=None,
        results_list=subject_results
    )

    # Task 6: Left vs Right Fists
    left_ids = [1, 3]
    right_ids = [2, 4]
    mask_lr = np.isin(y_all, left_ids + right_ids)
    y_leftright = np.where(np.isin(y_all[mask_lr], left_ids), 0, 1)
    evaluate_model(
        X_all[mask_lr], y_leftright,
        f"{subj_id}: Left vs Right Fists",
        save_dir=save_plot_dir,
        prefix=f"{subj_id}_Left_vs_Right_Fists",
        subject_id=subj_id,
        run_num=None,
        results_list=subject_results
    )


Task: S001: Real vs Imagined Fists | Acc: 0.56 | Prec: 0.54 | Rec: 0.54 | F1: 0.54
Task: S001: Left vs Right Fists | Acc: 0.56 | Prec: 0.54 | Rec: 0.54 | F1: 0.54
Task: S002: Real vs Imagined Fists | Acc: 0.67 | Prec: 0.62 | Rec: 0.77 | F1: 0.69
Task: S002: Left vs Right Fists | Acc: 0.52 | Prec: 0.50 | Rec: 0.38 | F1: 0.43
Task: S003: Real vs Imagined Fists | Acc: 0.52 | Prec: 0.50 | Rec: 0.62 | F1: 0.55
Task: S003: Left vs Right Fists | Acc: 0.63 | Prec: 0.62 | Rec: 0.62 | F1: 0.62
Task: S004: Real vs Imagined Fists | Acc: 0.52 | Prec: 0.50 | Rec: 0.69 | F1: 0.58
Task: S004: Left vs Right Fists | Acc: 0.52 | Prec: 0.50 | Rec: 0.77 | F1: 0.61
Task: S005: Real vs Imagined Fists | Acc: 0.70 | Prec: 0.65 | Rec: 0.85 | F1: 0.73
Task: S005: Left vs Right Fists | Acc: 0.63 | Prec: 0.75 | Rec: 0.43 | F1: 0.55
Task: S006: Real vs Imagined Fists | Acc: 0.48 | Prec: 0.43 | Rec: 0.23 | F1: 0.30
Task: S006: Left vs Right Fists | Acc: 0.52 | Prec: 0.50 | Rec: 0.46 | F1: 0.48
Task: S007: Real vs Im

In [7]:
# Combine and save all results
df = pd.DataFrame(run_results + subject_results)
csv_path = os.path.join(base_dir, "classification_metrics_summary.csv")
df.to_csv(csv_path, index=False)
print(f"Metrics saved to: {csv_path}")
df.head()


Metrics saved to: E:\AMAR\git\EEG_DATAREAD\edf_analyser\preprocessed_data_1005_multilabels\classification_metrics_summary.csv


,Subject,Run,Task,Accuracy,Precision,Recall,F1
0,S001,3.0,S001 Run 3: Real Left vs Real Right,0.4,0.3333,0.5000,0.4
1,S001,4.0,S001 Run 4: Imagined Left vs Imagined Right,0.4,0.3333,0.5000,0.4
2,S001,7.0,S001 Run 7: Real Left vs Real Right,0.4,0.3333,0.5000,0.4
3,S001,8.0,S001 Run 8: Imagined Left vs Imagined Right,0.4,0.0000,0.0000,0.0
4,S001,11.0,S001 Run 11: Real Left vs Real Right,0.8,1.0000,0.6667,0.8
